In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import defaultdict

In [2]:
def get_dfs(path="../streamlit/data"):
    properties_dfs = defaultdict(lambda: pd.DataFrame())
    split_df = None
    for path in Path(path).iterdir():
        res = re.match(r"(((?P<type>[a-z]+)_\d\.pkl)|(?P<csv>[a-z_]+.csv))", path.name.split("\\")[-1])
        if type_ := res.group("type"):
            properties_dfs[type_] = pd.concat([properties_dfs[type_], pd.read_pickle(path)])
        elif res.group("csv"):
            split_df = pd.read_csv(path, names=["split", "s1", "s2_cloudFree", "s2_cloudy", "name"])
    assert split_df is not None
    assert len(properties_dfs) == 3
    return properties_dfs, split_df
properties_dfs, split_df = get_dfs()

In [13]:
properties_dfs['cloudless']['name'] = properties_dfs['cloudless'][['roi', 'season', 'scene', 'patch']].apply(lambda x: "{}_{}_{}_p{}.tif".format(*x), axis=1).astype(str)

In [20]:
split_df['name'] = split_df['name'].astype(str)

In [1]:
reg = re.compile(r"(?P<roi>[A-Za-z\d]+)_(?P<season>[a-z]+)_(?P<scene>[\d]+)_p(?P<patch>\d+)\.tif")

def match_name(reg):
    def apply(x):
        return reg.match(x).groups()
    return apply

roi, season, scene, patch = match_name(reg)(split_df['name'].iloc[0])

NameError: name 're' is not defined

In [59]:
split_df['name'].apply(match_name(reg)).apply(lambda x: get_row(properties_dfs['cloudless'], *x))

In [52]:
def get_row(df, roi, season, scene, patch):
    res = df[(df['roi'] == roi) & (df['season'] == season) & (df['scene'] == scene) & (df['patch'] == patch)]
    if res.shape[0] == 0:
        return None
    else:
        return res.iloc[0]

get_row(properties_dfs['cloudless'], roi, season, scene, patch)

In [36]:
split_df['name'].iloc[0].split('')

['ROIs1158', 'spring', '101', 'p675.tif']

In [37]:
split_df['name']

0         ROIs1158_spring_101_p675.tif
1         ROIs1158_spring_101_p312.tif
2         ROIs1158_spring_101_p380.tif
3         ROIs1158_spring_101_p778.tif
4         ROIs1158_spring_101_p597.tif
                      ...             
118132     ROIs1158_spring_31_p215.tif
118133     ROIs1158_spring_31_p521.tif
118134     ROIs1158_spring_31_p494.tif
118135     ROIs1158_spring_31_p614.tif
118136     ROIs1158_spring_31_p212.tif
Name: name, Length: 118137, dtype: object

In [28]:
properties_dfs['cloudless'].isna().any().any()

False

In [21]:
properties_dfs['cloudless'].join(other=split_df, on='name')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [92]:
split_df.join(other=properties_dfs['cloudless'], on=['name'])

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat